# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key
output_data_file = "output_data/cities.csv"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
random_cit = pd.read_csv(output_data_file)
random_cit

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,coolum beach,-26.5333,153.1000,74.34,61,61,6.31,AU,1628223225
1,1,vilhena,-12.7406,-60.1458,67.87,46,77,2.17,BR,1628223227
2,2,challapata,-18.9000,-66.7667,39.67,31,0,3.76,BO,1628223228
3,3,hilo,19.7297,-155.0900,80.33,82,90,9.22,US,1628223213
4,4,pimentel,-6.8367,-79.9342,63.00,84,0,3.00,PE,1628223231
...,...,...,...,...,...,...,...,...,...,...
570,570,gornopravdinsk,60.0500,69.9000,69.22,66,94,5.23,RU,1628224176
571,571,masaka,-0.3338,31.7341,63.99,70,60,3.49,UG,1628224178
572,572,hay river,60.8156,-115.7999,57.18,82,40,5.75,CA,1628224179
573,573,fairhope,30.5230,-87.9033,80.35,90,40,0.00,US,1628224181


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)
location = random_cit[["Lat", "Lng"]]
humidity = random_cit["Humidity"]
maps = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights = humidity, dissipating = False, max_intensity = 300, point_radius = 5)
maps.add_layer(heat_layer)
maps

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
nice_temp = random_cit.loc[(random_cit["Max Temp"] < 80) & (random_cit["Max Temp"] > 70), :]
no_wind = nice_temp.loc[random_cit["Wind Speed"] < 10, :]
paradise = no_wind.loc[random_cit["Cloudiness"] == 0, :]
paradise


,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
76,76,semey,50.4111,80.2275,73.62,49,0,6.71,KZ,1628223359
177,177,gizel,43.0400,44.5708,70.02,83,0,2.24,RU,1628223531
211,211,flinders,-34.5833,150.8552,72.18,46,0,1.01,AU,1628223586
318,318,ulladulla,-35.3500,150.4667,70.88,49,0,7.16,AU,1628223757
332,332,kysyl-syr,63.8986,122.7617,76.91,28,0,4.81,RU,1628223779
391,391,batemans bay,-35.7167,150.1833,70.20,47,0,7.20,AU,1628223879
479,479,uruzgan,32.8333,66.0000,79.81,18,0,5.61,AF,1628224027
569,569,kars,40.6020,43.0950,70.65,82,0,2.30,TR,1628224175


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = paradise[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = " "
hotel_df

,City,Country,Lat,Lng,Hotel Name
76,semey,KZ,50.4111,80.2275,
177,gizel,RU,43.0400,44.5708,
211,flinders,AU,-34.5833,150.8552,
318,ulladulla,AU,-35.3500,150.4667,
332,kysyl-syr,RU,63.8986,122.7617,
391,batemans bay,AU,-35.7167,150.1833,
479,uruzgan,AF,32.8333,66.0000,
569,kars,TR,40.6020,43.0950,


In [117]:
base_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    params["location"] = f"{Lat}, {Lng}"
    response = requests.get(base_url, params=params)
    name_address = response.json()
    try:
         hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except(KeyError, IndexError):
         print("Error, sorry..")
hotel_df

Error, sorry..
Error, sorry..


,City,Country,Lat,Lng,Hotel Name
76,semey,KZ,50.4111,80.2275,Grand Hotel Semey
177,gizel,RU,43.0400,44.5708,Александрия
211,flinders,AU,-34.5833,150.8552,Shellharbour Resort & Conference Centre
318,ulladulla,AU,-35.3500,150.4667,Sandpiper Motel
332,kysyl-syr,RU,63.8986,122.7617,
391,batemans bay,AU,-35.7167,150.1833,Batemans Bay Marina Resort
479,uruzgan,AF,32.8333,66.0000,
569,kars,TR,40.6020,43.0950,Kars Otel


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
maps.add_layer(marker_layer)


# Display figure
maps

Figure(layout=FigureLayout(height='420px'))